LOADING DATA

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import multiprocessing
import functools

2023-02-11 03:30:34.048524: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 03:30:34.170138: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-11 03:30:34.170161: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-11 03:30:35.146038: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
def read_csv(img,data,lbl,rot,id):
    rows = open('dataset4_csv/ID_'+str(id)+'.csv').read().strip().split('\n')
    for row in rows:
        row = row.split(',')
        (x,y,label,rotation,pathimg) = row
        image =  load_img(pathimg, target_size=(256, 256))
        image = img_to_array(image)
        img.append(image)
        data.append([x,y])
        lbl.append(label)
        rot.append(rotation)

In [3]:
if __name__ == '__main__':
    
    manager = multiprocessing.Manager()
    img = manager.list()
    data = manager.list()
    lbl = manager.list()
    rot = manager.list()

    id = list(range(1,4))

    partial_read_csv = functools.partial(read_csv, img, data, lbl, rot)

    pool = multiprocessing.Pool()
    pool.map(partial_read_csv, id)

In [6]:
pool.close()
pool.terminate()

In [7]:
img = np.array(img,dtype='float32')
data = np.array(data,dtype='float32')
lbl = np.array(lbl)
rot = np.array(rot)
lb = LabelBinarizer()
lbl = lb.fit_transform(lbl)
rot = lb.fit_transform(rot)

In [8]:
split = train_test_split(img, lbl, rot, data, test_size=0.1)
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainRotations, testRotations) = split[4:6]
(trainCoordinates, testCoordinates) = split[6:8]
split = train_test_split(trainImages, trainLabels, trainRotations, trainCoordinates, test_size=0.1)
(trainImages, valImages) = split[:2]
(trainLabels, valLabels) = split[2:4]
(trainRotations, valRotations) = split[4:6]
(trainCoordinates, valCoordinates) = split[6:8]

MODEL CREATION

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Rescaling, Dropout, BatchNormalization

In [12]:
#input layer
inputs = Input(shape=(256,256,3))
scaled_inputs = Rescaling(1/255)(inputs)

#label branch
l = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(scaled_inputs)
l = BatchNormalization()(l)
l = MaxPooling2D()(l)
l = Dropout(0.2)(l)
l = Conv2D(32, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(l)
l = BatchNormalization()(l)
l = MaxPooling2D()(l)
l = Dropout(0.3)(l)
l = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(l)
l = BatchNormalization()(l)
l = MaxPooling2D()(l)
l = Dropout(0.4)(l)
l = Flatten()(l)
l = Dense(128, activation='relu', kernel_initializer='he_uniform')(l)
l = BatchNormalization()(l)
l = Dropout(0.5)(l)
l = Dense(3, activation = 'softmax', name='label')(l)

#rot branch
r = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(scaled_inputs)
r = BatchNormalization()(r)
r = MaxPooling2D()(r)
r = Dropout(0.2)(r)
r = Conv2D(32, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(r)
r = BatchNormalization()(r)
r = MaxPooling2D()(r)
r = Dropout(0.3)(r)
r = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(r)
r = BatchNormalization()(r)
r = MaxPooling2D()(r)
r = Dropout(0.4)(r)
r = Flatten()(r)
r = Dense(128, activation='relu', kernel_initializer='he_uniform')(r)
r = BatchNormalization()(r)
r = Dropout(0.5)(r)
r = Dense(15, activation = 'softmax', name='rotation')(r)

#ann branch
a = Conv2D(16, (3,3), 1, activation = 'relu')(scaled_inputs)
a = MaxPooling2D()(a)
a = Conv2D(32, (3,3), 1, activation = 'relu')(a)
a = MaxPooling2D()(a)
a = Conv2D(64, (3,3), 1, activation = 'relu')(a)
a = MaxPooling2D()(a)
a = Conv2D(128, (3,3), 1, activation = 'relu')(a)
a = MaxPooling2D()(a)
a = Flatten()(a)
a = Dense(32, activation = 'linear')(a)
a = Dense(32, activation = 'linear', kernel_regularizer='l2', bias_regularizer='l2')(a)
a = Dense(32, activation = 'linear', kernel_regularizer='l2', bias_regularizer='l2')(a)
a = Dense(32, activation = 'linear')(a)
a = Dense(2, activation = 'linear', name='coordinate')(a)

In [13]:
#model
model = Model(
	inputs=inputs,
	outputs=(l, r, a))

#compile
model.compile(loss={
	'label': 'categorical_crossentropy',
	'rotation': 'categorical_crossentropy',
	'coordinate': 'mse',
}, optimizer='adam', metrics=['accuracy'])

hist = model.fit(
	trainImages, {
	'label': trainLabels,
	'rotation': trainRotations,
	'coordinate': trainCoordinates,
},
	validation_data= (valImages, {
	'label': valLabels,
	'rotation': valRotations,
	'coordinate': valCoordinates,
}),
	batch_size=32,
	epochs=20)

2023-02-11 03:34:07.722647: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 4643094528 exceeds 10% of free system memory.


Epoch 1/20
185/185 [==============================] - 308s 2s/step - loss: 2.1101 - label_loss: 0.5979 - rotation_loss: 1.2253 - coordinate_loss: 0.0011 - label_accuracy: 0.8018 - rotation_accuracy: 0.6567 - coordinate_accuracy: 0.8010 - val_loss: 3.9957 - val_label_loss: 1.3597 - val_rotation_loss: 2.5521 - val_coordinate_loss: 4.7699e-04 - val_label_accuracy: 0.3409 - val_rotation_accuracy: 0.1218 - val_coordinate_accuracy: 0.8554
Epoch 2/20
185/185 [==============================] - 267s 1s/step - loss: 0.7592 - label_loss: 0.2537 - rotation_loss: 0.4731 - coordinate_loss: 4.8091e-04 - label_accuracy: 0.9094 - rotation_accuracy: 0.8596 - coordinate_accuracy: 0.8606 - val_loss: 2.8639 - val_label_loss: 1.0214 - val_rotation_loss: 1.8355 - val_coordinate_loss: 4.1282e-04 - val_label_accuracy: 0.5312 - val_rotation_accuracy: 0.4338 - val_coordinate_accuracy: 0.8676
Epoch 3/20
185/185 [==============================] - 276s 1s/step - loss: 0.5064 - label_loss: 0.1737 - rotation_loss: 0.

KeyboardInterrupt: 

In [ ]:
model.save('super2.h5')

TESTING

In [ ]:
import numpy as np
import cv2
from keras.models import load_model
from matplotlib import pyplot as plt
from keras.utils.vis_utils import plot_model

In [ ]:
model = load_model('super1.h5', compile=False)
model.compile(loss={
	'label': 'categorical_crossentropy',
	'rotation': 'categorical_crossentropy',
	'coordinate': 'mse',
}, optimizer='adam', metrics=['accuracy'])

In [ ]:
plot_model(model)

In [ ]:
image = cv2.imread('test_images/96trans50_-150.jpg')
image_show = np.array(image, dtype='float32')
height, width = image_show.shape[:2]
image_input = cv2.resize(image_show, (256,256))
plt.imshow(image_show)

In [ ]:
y = model.predict(np.expand_dims(image_input,0))
labels = ['ID_1', 'ID_2', 'ID_3']
rotations = [0,120,144,168,192,216,24,240,264,288,312,336,48,72,96]
label = y[0].argmax()
rotation = y[1].argmax()
co_x = y[2][0][0]
co_y = y[2][0][1]
cv2.circle(image_show, (int(np.around(co_x*width+width/2)),int(np.around(co_y*height+height/2))), 190, (255,255,255), thickness=2)
cv2.putText(image_show, str(labels[label]) + ' ' + str(rotations[rotation]) + ' degree', (100,100), cv2.FONT_HERSHEY_TRIPLEX, 4, (255,255,255), thickness=2)
plt.imshow(image_show)